In [1]:
import os
import openai
import streamlit as st
import pandas as pd
from llama_index import StorageContext, load_index_from_storage
from llama_index import (
    Document,
    GPTVectorStoreIndex,
    LLMPredictor,
    PromptHelper,
    ServiceContext,
    LangchainEmbedding
)

from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index.prompts.prompts import QuestionAnswerPrompt

# APIキーなどの設定
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = 'azure'
openai.api_version = '2023-05-15'

In [2]:
#service_contextの生成
def create_service_context():
    # LLM Predictor
    llm_predictor = LLMPredictor(llm=AzureChatOpenAI(
        deployment_name='GPT35TURBO',         #デプロイ名
        max_tokens=3000,                        #最大トークン数
        temperature=1,                          #出力のランダム度合い
        openai_api_version=openai.api_version   #openaiのapiのバージョン情報
    ))

    # テキストの埋め込みベクトル変換(Embedding)に関する設定
    embeddings = LangchainEmbedding(OpenAIEmbeddings(
        engine="ADA",        #エンベディングに使うモデル
        chunk_size=1,                           #ここでのチャンクサイズはバッチサイズ
        openai_api_version=openai.api_version   #openaiのapiのバージョン情報
    ))

    # Prompt Helper（テキスト分割に関する設定）
    prompt_helper = PromptHelper(
        max_input_size=3000,    # 最大入力サイズ
        num_output=1000,        # LLMの出力サイズ
        chunk_size_limit=1000,  # 使用する最大チャンクサイズ（チャンク：テキストを細かく分割したもの）
        max_chunk_overlap=0,    # チャンクオーバーラップの最大トークン数
        separator="。"        # テキスト分割の区切り文字
    )

    # Service Contextの生成
    service_context = ServiceContext.from_defaults(
        llm_predictor=llm_predictor,    # LLM Predictor
        embed_model=embeddings,         # エンベディングについての設定
        prompt_helper=prompt_helper     # Prompt Helper
    )

    return service_context, prompt_helper

def llama_generate(index, query, top_k):
    """llama-indexによる回答の生成"""
    # 与えるコンテキスト（商品リストのうちクエリとの類似度が高いもの）をもとに回答をもとめるようなプロンプト
    QA_PROMPT_TMPL = (
        "私たちは以下の情報をコンテキスト情報として与えます。 \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "この情報をもとに質問に日本語で回答してください。: {query_str}\n"
    )
    qa_prompt = QuestionAnswerPrompt(QA_PROMPT_TMPL)
    
    # 回答生成
    
    # プロンプトと上位いくつまでの類似度を使用するか設定
    query_engine = index.as_query_engine(
        engine='GPT35TURBO',#←ここを変更
        text_qa_template=qa_prompt, # 上記のプロンプトを与える（デフォルトは英語文）
        similarity_top_k=top_k      # 参考情報（商品リスト）のうちクエリとの類似度上位何件を生成に利用するか
    )
    # 生成
    response = query_engine.query(query)

    return response

In [8]:
service_context, prompt_helper = create_service_context()
storage_context = StorageContext.from_defaults(persist_dir="./storage_pandas")


In [9]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [10]:
index

In [11]:
# クエリ （description：アップロードした顧客情報)
query = f"""# Background
            You are an expert in the field of data collection in data science.
            Your job is to use your data collection expertise in data science to support user learning. 
            However, you can also enjoy stories other than data science.

            # Customer Info
            initial_prompt: あなたはデータサイエンスのデータ収集分野におけるスペシャリストです。
データ収集の分野でユーザーをサポートしてください。
            User request: pandasについて教えてください。
            # Instructions
            データサイエンスに関する質問をされた場合、そのプロセスを最適化するために、ユーザーをサポートしてください。
            具体的には、User requestを解決するライブラリ名や選定理由、コードと使用方法、ベストプラクティスなどを含めてください。
            ユーザーの要求に対して役立つコードを提示し、説明をセットで
            詳しく説明することを意識してください。ライブラリを使用するために必要なimport文、基本的な使用方法から実戦的な使い方まで順を追って説明してください。
            コードは見やすくするために、他の文章と続けて文章にせず、コードとして区別してください。
            コードを書いた場合には、そのコードの実行結果として出力を必ずセットで提供してください。
                        """

# llama-indexによる回答の生成
result = llama_generate(index=index, query=query, top_k=1)

In [12]:
result

Response(response='pandasについて教えてください。\n\npandasは、Pythonにおけるデータ分析のためのライブラリであり、特に表形式のデータを扱う際に便利です。その中のDataFrame.dropnaメソッドは、欠損値を含む行や列を削除するためのメソッドです。\n\nまず、このメソッドは以下のように使うことができます。\n\n```python\nimport pandas as pd\n\n# サンプルのDataFrameを作成\ndf = pd.DataFrame({\n    "name": [\'Alfred\', \'Batman\', \'Catwoman\'],\n    "toy": [np.nan, \'Batmobile\', \'Bullwhip\'],\n    "born": [pd.NaT, pd.Timestamp("1940-04-25"), pd.NaT]\n})\n\n# 欠損値を含む行を削除\nresult = df.dropna()\nprint(result)\n# 出力結果\n#    name        toy       born\n# 1  Batman  Batmobile 1940-04-25\n```\n\nまた、特定の軸（行または列）、特定の条件（欠損値が一つでも含まれるか、全てが欠損値か）、特定の非欠損値の数などに基づいて、削除の方法を細かく指定することもできます。\n\n例えば、特定の列（カラム）に欠損値が含まれている場合にその列を削除する場合は、次のようにします。\n\n```python\n# 欠損値を含む列を削除\nresult = df.dropna(axis=\'columns\')\nprint(result)\n# 出力結果\n#      name\n# 0  Alfred\n# 1  Batman\n# 2 Catwoman\n```\n\nこのように、pandasのDataFrame.dropnaメソッドは、データの前処理やクリーニングにおいて非常に便利なメソッドです。', source_nodes=[NodeWithScore(node=Node(text='【Name】Pandas【Chapter】DataFrame【Section】pandas